In [25]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [26]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [28]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True
                                           )

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor()
                                          )

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           num_workers=2
                                           )

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          num_workers=2
                                          )


In [37]:
# bidirectional_recurrent_neural_network (many to one)
# differences with one directional LSTM :
# 1) "bidirectional=True" in LSTM
# 2) "hidden size * 2" for bidirection in fc
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.classifer = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        # set initial cell and hidden states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

        # forward
        # output: (batch_size, seq_length, hidden_size*2)
        # "_": ( h(t), c(t) )
        out, _ = self.lstm(x, (h0, c0))              

        # decode the hidden state of the last time step
        out = self.classifer(out[:, -1, :])
        return out

In [38]:
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
# train model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5790
Epoch [1/2], Step [200/600], Loss: 0.2430
Epoch [1/2], Step [300/600], Loss: 0.2948
Epoch [1/2], Step [400/600], Loss: 0.2016
Epoch [1/2], Step [500/600], Loss: 0.1079
Epoch [1/2], Step [600/600], Loss: 0.1404
Epoch [2/2], Step [100/600], Loss: 0.1618
Epoch [2/2], Step [200/600], Loss: 0.1636
Epoch [2/2], Step [300/600], Loss: 0.1066
Epoch [2/2], Step [400/600], Loss: 0.0547
Epoch [2/2], Step [500/600], Loss: 0.1344
Epoch [2/2], Step [600/600], Loss: 0.0663


In [40]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predict = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predict == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
# torch.save(model.state_dict(), 'brnn.pkl') 

Test Accuracy of the model on the 10000 test images: 97.95 %
